In [1]:
############################################
# 0. CONFIG
############################################
MODEL_NAME = "FacebookAI/roberta-large"
DATA_PATH  = "/kaggle/input/foodhazard/chunked_deberta_512.json"

############################################
# 1. IMPORTS
############################################
import json, os, torch, pandas as pd
import numpy as np
from torch import nn
import torch.nn.functional as F
from datasets import Dataset, Features, Value
from transformers import (
    AutoTokenizer, AutoModel, TrainingArguments, Trainer
)
from sklearn.metrics import precision_recall_fscore_support

os.environ["WANDB_DISABLED"] = "true"



2025-12-04 02:45:45.931057: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764816346.138533      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764816346.202280      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [2]:
############################################
# 2. FOCAL LOSS
############################################
class FocalLoss(nn.Module):
    def __init__(self, alpha=1.0, gamma=2.0):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, inputs, targets):
        """
        inputs: [B, C]
        targets: [B]
        """
        ce_loss = F.cross_entropy(inputs, targets, reduction="none")
        pt = torch.exp(-ce_loss)
        loss = self.alpha * (1 - pt) ** self.gamma * ce_loss
        return loss.mean()

############################################
# 3. LOAD DATA
############################################
with open(DATA_PATH, "r", encoding="utf-8") as f:
    raw = json.load(f)
df = pd.DataFrame(raw)

# text field: ưu tiên chunk_text, fallback sang text
if "chunk_text" in df.columns:
    df["text"] = df["chunk_text"]
elif "merged_text" in df.columns:
    df["text"] = df["merged_text"]
else:
    df["text"] = df["text"]

# encode product & hazard labels
df["product_labels"] = df["product_category"].astype("category").cat.codes
df["hazard_labels"]  = df["hazard_category"].astype("category").cat.codes

N_PRODUCT = df["product_category"].nunique()
N_HAZARD  = df["hazard_category"].nunique()
print("N_PRODUCT:", N_PRODUCT, "| N_HAZARD:", N_HAZARD)

############################################
# 4. MODEL: MULTI-TASK
############################################
class MultiTaskClassifier(nn.Module):
    def __init__(self, base_model_name, n_product, n_hazard):
        super().__init__()
        self.backbone = AutoModel.from_pretrained(base_model_name)
        hidden_size = self.backbone.config.hidden_size

        self.dropout = nn.Dropout(0.1)
        self.product_head = nn.Linear(hidden_size, n_product)
        self.hazard_head  = nn.Linear(hidden_size, n_hazard)

        self.focal = FocalLoss()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        product_labels=None,
        hazard_labels=None,
        **kwargs
    ):
        outputs = self.backbone(
            input_ids=input_ids,
            attention_mask=attention_mask,
        )
        cls = outputs.last_hidden_state[:, 0]  # [CLS]
        cls = self.dropout(cls)

        product_logits = self.product_head(cls)
        hazard_logits  = self.hazard_head(cls)

        loss = None
        if (product_labels is not None) and (hazard_labels is not None):
            loss_product = self.focal(product_logits, product_labels)
            loss_hazard  = self.focal(hazard_logits, hazard_labels)
            loss = 0.5 * loss_product + 0.5 * loss_hazard

        # Trainer cần "logits" để không crash, ta concat hai head
        logits_concat = torch.cat([product_logits, hazard_logits], dim=-1)

        return {
            "loss": loss,
            "logits": logits_concat,          # Trainer dùng cái này
            "product_logits": product_logits, # để sau này infer tay
            "hazard_logits": hazard_logits,
        }

############################################
# 5. DATASET + TOKENIZER
############################################
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_function(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        padding="max_length",
        max_length=512,
    )

features = Features({
    "text": Value("string"),
    "product_labels": Value("int64"),
    "hazard_labels": Value("int64"),
})

dataset = Dataset.from_pandas(
    df[["text", "product_labels", "hazard_labels"]],
    features=features
)

ds = dataset.train_test_split(test_size=0.2, seed=42)
tokenized = ds.map(tokenize_function, batched=True)

############################################
# 6. (TẠM THỜI) KHÔNG COMPUTE_METRICS TRONG TRAINER
############################################
# Nếu muốn giữ khung:
# def compute_metrics(pred):
#     return {}

############################################
# 7. INIT MODEL
############################################
model = MultiTaskClassifier(
    base_model_name=MODEL_NAME,
    n_product=N_PRODUCT,
    n_hazard=N_HAZARD,
)

############################################
# 8. TRAINING ARGS 
############################################
args = TrainingArguments(
    output_dir="./result_multi_task",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=4,

    learning_rate=1e-5,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    weight_decay=0.01,
    num_train_epochs=10,
    fp16=True,

    eval_strategy="epoch",     
    save_strategy="epoch",
    do_eval=True,
    load_best_model_at_end=True,

    metric_for_best_model="eval_loss",  
    greater_is_better=False,

    save_total_limit=2,
)

############################################
# 9. TRAINER
############################################
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
    tokenizer=tokenizer,
)

trainer.train()
print("DONE TRAINING 🚀")


N_PRODUCT: 22 | N_HAZARD: 10


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at FacebookAI/roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipykernel_19/1312295209.py:165: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,1.624600,0.640366
2,0.497700,0.460501
3,0.325500,0.407913
4,0.231800,0.417106
5,0.167100,0.418041
6,0.114400,0.441116
7,0.098700,0.462146
8,0.068000,0.475880
9,0.056500,0.483103
10,0.049800,0.483206


DONE TRAINING 🚀
